In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv("news_dataset.csv")
data = data.sample(500)
data.head()

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,text_without_stopwords,hasImage
1234,Infowars Nightly News,2016-10-28T22:37:02.718+03:00,clinton camp desperate russia trains for wwiii,britain should embrace unilateral free trade r...,english,infowars.com,http://hw.infowars.com/wp-content/uploads/2016...,conspiracy,Fake,clinton inc watch pains msm report clinton cor...,hillary indicted election trump responds fbi i...,1.0
671,No Author,2016-11-16T22:46:57.325+02:00,life chaos this coffee shop is also an art gal...,ever wonder whats on the mind of todays most n...,english,clickhole.com,http://images.onionstatic.com/clickhole/3467/3...,satire,Fake,life chaos coffee shop also art gallery,ever wonder whats mind todays notable people w...,1.0
1724,André du Pôle,2016-10-27T20:01:14.275+03:00,more emotions men should master,gentlemen donald j trump is the new presidente...,english,returnofkings.com,No Image URL,hate,Real,comment reasons diet sucks important concepts ...,rejoice gentlemen tears liberals flowing freel...,0.0
478,Alex Ansary,2016-11-07T20:01:08.838+02:00,janet reno first female us attorney general di...,las vegas may become the new amsterdam \r\nhi...,english,amtvmedia.com,http://www.amtvmedia.com/wp-content/uploads/20...,bs,Fake,janet reno first female us attorney general dies,las vegas may become new amsterdam high times ...,1.0
375,Harun Yahya,2016-11-17T02:00:00.000+02:00,crossing the line in democracy,shares\r\n \r\nleader at alhashd alshaabi i...,english,ahtribune.com,http://ahtribune.com/images/media/Taksim_Squar...,bs,Fake,crossing line democracy,shares leader alhashd alshaabi hawija district...,1.0


In [ ]:
#text-based features
#word count
import nltk
import string
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

def word_count(text):
    text = str(text).lower()
    text = text.replace("\r\n", ' ')
    if text == "no title":
        return 0
    else:
        return len(str(text).split(' '))

def sentence_count(text):
    sentences = nltk.sent_tokenize(text)
    return len(sentences)

def average_word_length(text):
    words = text.split()
    count = 0
    for word in words:
        count += len(word)
    return count / len(words)
def punctuation_count(text):
    count = 0
    for word in text:
        if word in string.punctuation:
            count += 1
    return count

def stopword_count(text):
    stopword = stopwords.words('english')
    count = 0
    for word in text.split():
        if word in stopword:
            count += 1
    return count

#change data type to string
data['text'] = data['text'].astype(str)
data['title'] = data['title'].astype(str)

data['text_word_count'] = data['text'].apply(word_count)
data['title_word_count'] = data['title'].apply(word_count)

data['text_sentence_count'] = data['text'].apply(sentence_count)
data['title_sentence_count'] = data['title'].apply(sentence_count)

data['text_average_word_length'] = data['text'].apply(average_word_length)
data['title_average_word_length'] = data['title'].apply(average_word_length)

data['text_punctuation_count'] = data['text'].apply(punctuation_count)
data['title_punctuation_count'] = data['title'].apply(punctuation_count)

data['text_stopwords_count'] = data['text'].apply(stopword_count)
data['title_stopwords_count'] = data['title'].apply(stopword_count)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yj_li\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yj_li\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
data

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,...,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count
1234,Infowars Nightly News,2016-10-28T22:37:02.718+03:00,clinton camp desperate russia trains for wwiii,britain should embrace unilateral free trade r...,english,infowars.com,http://hw.infowars.com/wp-content/uploads/2016...,conspiracy,Fake,clinton inc watch pains msm report clinton cor...,...,1826,7,1,1,4.893761,5.714286,0,0,827,1
671,No Author,2016-11-16T22:46:57.325+02:00,life chaos this coffee shop is also an art gal...,ever wonder whats on the mind of todays most n...,english,clickhole.com,http://images.onionstatic.com/clickhole/3467/3...,satire,Fake,life chaos coffee shop also art gallery,...,136,10,1,1,4.323077,4.100000,0,0,58,3
1724,André du Pôle,2016-10-27T20:01:14.275+03:00,more emotions men should master,gentlemen donald j trump is the new presidente...,english,returnofkings.com,No Image URL,hate,Real,comment reasons diet sucks important concepts ...,...,1135,5,1,1,5.185817,5.400000,0,0,460,2
478,Alex Ansary,2016-11-07T20:01:08.838+02:00,janet reno first female us attorney general di...,las vegas may become the new amsterdam \r\nhi...,english,amtvmedia.com,http://www.amtvmedia.com/wp-content/uploads/20...,bs,Fake,janet reno first female us attorney general dies,...,750,9,1,1,5.286704,4.777778,0,0,278,1
375,Harun Yahya,2016-11-17T02:00:00.000+02:00,crossing the line in democracy,shares\r\n \r\nleader at alhashd alshaabi i...,english,ahtribune.com,http://ahtribune.com/images/media/Taksim_Squar...,bs,Fake,crossing line democracy,...,303,5,1,1,5.263699,5.200000,0,0,94,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,recrisp,2016-10-27T02:54:50.332+03:00,little blue balls,russia puzzled by norways decision to host us ...,english,abovetopsecret.com,http://files.abovetopsecret.com/images/ats7.jpg,bs,Fake,little blue balls,...,244,3,1,1,4.589212,5.000000,0,0,114,0
94,Anonymous,2016-10-29T22:36:00.000+03:00,no title,the second video bottom one proves what trump...,english,abeldanger.net,No Image URL,bs,Fake,title,...,49,0,1,1,7.146341,3.500000,0,0,18,1
1588,No Author,2016-10-27T23:04:00.000+03:00,bayern munich beat augsburg,truth revolt october \r\nas usual pat condel...,english,presstv.ir,http://media.presstv.com/photo/20161027/d014e7...,state,Real,two izadi women escaped daesh win eus top righ...,...,64,4,1,1,5.403509,6.000000,0,0,19,0
1734,Kshatriya Indransha,2016-10-30T21:00:00.000+02:00,how population growth industrial agriculture a...,buzz aldrin is a warning of how success can le...,english,returnofkings.com,No Image URL,hate,Real,elites using divide rule control us,...,3095,12,1,1,4.649269,7.250000,0,0,1467,3
